In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append(os.path.dirname(os.path.abspath('')))
#from utils.spark import restart_spark
from stratification.params import SplitBuilderParams
from prepilot.params import PrepilotParams
from prepilot.prepilot_experiment_builder import PrepilotExperimentBuilder
from prepilot.prepilot_split_builder import PrepilotSplitBuilder
from auto_ab.abtest import ABTest
from auto_ab.params import ABTestParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2
from stratification.split_builder import StratificationSplitBuilder

# Кейс №6. Выделение групп

In [5]:
df = pd.read_csv('ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [19]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 10000,
        'target1': 10000
    },
    region_col = "moda_city",
    split_metric_col = "height_now",
    customer_col = "id",
    cols = [],
    cat_cols=[
        
    ],
    pvalue=0.05,
    n_top_cat=100,
    stat_test="ttest_ind"
)

In [20]:
split_builder = StratificationSplitBuilder(df, split_builder_params)

In [21]:
guests_data_with_strata = split_builder.assign_strata()

INFO:stratification.split_builder:Calculate stratas for guest table
INFO:stratification.binning:27222 outliers found


In [22]:
guests_data_with_strata.head()

,Unnamed: 0,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,strata
0,0,172.470442,166.382868,163.648256,173.422115,5.644384,-1.923929,A,0,3,1,3017
12,12,172.760235,168.417151,165.007237,175.096232,8.241634,-0.112896,B,12,3,2,3017
60,60,172.549884,170.020950,167.768344,174.927131,5.166898,11.724892,A,60,3,1,3017
63,63,171.329783,169.439431,172.125136,173.563411,7.784052,1.632047,A,63,3,2,3017
66,66,170.706926,168.760224,162.454053,171.930235,3.629899,-3.041150,B,66,3,2,3017


In [23]:
groups = split_builder.build_split(guests_data_with_strata)

INFO:stratification.split_builder:control: Desired size = 0.05,             resulting size = 9997, diff = 0.03 %
INFO:stratification.split_builder:target: Desired size = 0.052630747935558914,             resulting size = 9997, diff = 0.03 %
INFO:stratification.split_builder:target1: Desired size = 0.05555370376543004,             resulting size = 9982, diff = 0.18 %
/Users/egorshishkovets/opt/anaconda3/lib/python3.9/site-packages/ab_tools-0.1.0-py3.9.egg/stratification/stat_test.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/Users/egorshishkovets/opt/anaconda3/lib/python3.9/site-packages/ab_tools-0.1.0-py3.9.egg/stratification/stat_test.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
INFO:stratification.split_builder:Success!


In [24]:
groups

,Unnamed: 0,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,strata,group_name
73,73,167.326851,165.613285,162.921686,170.023141,4.742967,3.894028,A,73,3,2,30-1,target1
239,239,165.050701,161.898954,157.990947,167.990884,-3.123070,4.237944,B,239,3,1,301,control
324,324,172.662366,168.168034,167.593891,173.738999,4.589677,2.018746,B,324,3,2,3017,target
371,371,164.765206,159.577823,159.846411,166.894030,1.511456,0.918961,B,371,3,2,30-1,target1
559,559,171.905814,168.422144,166.469087,173.377120,13.689848,2.389145,A,559,3,1,3017,target
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199774,199774,174.873666,172.286212,173.524213,178.512743,2.460086,4.442295,A,199774,4,2,4118,control
199809,199809,176.889932,174.443774,176.407333,178.223087,-0.316250,6.903280,A,199809,4,1,411,target1
199829,199829,174.780901,169.967739,171.542452,177.126472,4.176728,5.784473,B,199829,4,1,4119,target
199889,199889,176.472726,177.142132,171.343139,179.917180,11.504313,3.765237,B,199889,4,2,414,control
